In [3]:
from Bio import SeqIO
import pandas as pd
import torch
from tape import ProteinBertModel, TAPETokenizer
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import torch
from tape import ProteinBertModel, TAPETokenizer
model = ProteinBertModel.from_pretrained('bert-base')
tokenizer = TAPETokenizer(vocab='iupac')

lookup_d = {}
for record in SeqIO.parse("deeploc_data.fasta", "fasta"):
    description = record.description
    seq = record.seq
    desc_split = description.split(" ")
    if len(desc_split) == 3:
        ID = desc_split[0]
        label = desc_split[1]
        label = label[:len(label)-2]
        lookup_d[ID] = label
    
    else:
        ID = desc_split[0]
        label = desc_split[1]
        label = label[:len(label)-2]
        lookup_d[ID] = label


In [4]:
import numpy as np
import pandas as pd
import json
arrays = np.load('output_full.npz', allow_pickle=True)

l = list(arrays.keys()) 

In [5]:
deeploc_df = pd.DataFrame()
label_df = pd.DataFrame(columns=['ID', 'label'])
for a in l:
    d = arrays[a].item()['avg']
    df = pd.DataFrame(d)
    label_df = label_df.append({"ID":a, "label": lookup_d[a]}, ignore_index=True)
    deeploc_df = deeploc_df.append(df.transpose(), ignore_index=True)

In [8]:
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
# tsne = TSNE(n_components=3)
pca = PCA(n_components=3)

principalComponents = pca.fit_transform(deeploc_df)
# tsneComponents = tsne.fit_transform(deeploc_df)

In [9]:
principal_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2', 'pc3'])

# tsne_df = pd.DataFrame(data=tsneComponents, columns=['tsne1', 'tsne2', 'tsne3'])

In [10]:
principal_df['ID'] = label_df['ID']
principal_df['label'] = label_df['label']

# tsne_df['ID'] = label_df['ID']
# tsne_df['label'] = label_df['label']


In [11]:
principal_df

,pc1,pc2,pc3,ID,label
0,-3.785695,3.183419,3.369541,P83585,Plastid
1,-3.962483,3.689283,4.857982,P12100,Extracellular
2,-2.999231,-0.460258,1.221900,Q8GUU3,Nucleus
3,-3.201531,1.562028,3.228625,P25379,Mitochondrion
4,-3.467626,-0.842392,2.771395,P19882,Mitochondrion
...,...,...,...,...,...
6189,0.125758,-0.585828,-1.191705,Q9FMD7,Cell.membrane
6190,2.077721,-2.494235,0.908155,Q9LDN0,Cytoplasm
6191,5.739468,2.096906,0.103102,Q8N7E2,Cytoplasm
6192,-2.710245,1.903203,1.856477,P85971,Cytoplasm


In [12]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = list(label_df['label'].unique())
colors =px.colors.qualitative.G10
for target, color in zip(targets,colors):
    indicesToKeep = principal_df['label'] == target
    ax.scatter(principal_df.loc[indicesToKeep, 'pc1']
               , principal_df.loc[indicesToKeep, 'pc2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()
plt.show()

NameError: name 'px' is not defined

In [ ]:
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
import plotly.express as px
plt.figure(figsize=(25, 25))
fig = px.scatter(principal_df, x='pc1', y='pc2',
              color='label', color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

In [ ]:
import plotly.express as px
plt.figure(figsize=(1800,1800))
fig = px.scatter_3d(principal_df, x='pc1', y='pc2', z='pc3',
              color='label', color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = list(label_df['label'].unique())
print(len(targets))
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k', '0.25', '0.5', '0.75', '0.9']
for target, color in zip(targets,colors):
    indicesToKeep = tsne_df['label'] == target
    ax.scatter(tsne_df.loc[indicesToKeep, 'tsne1']
               , tsne_df.loc[indicesToKeep, 'tsne2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
import plotly.express as px
plt.figure(figsize=(25, 25))
fig = px.scatter_3d(tsne_df, x='tsne1', y='tsne2', z='tsne3',
              color='label', color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

In [ ]:
2